<a href="https://colab.research.google.com/github/ObaidAshraf/GenAI_AgenticAI/blob/master/rag_langchain_pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain langchain-pinecone langchain-google-genai pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.2/342.2 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 16.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curren

In [ ]:
import os

from google.colab import userdata
pinecone_api_key = userdata.get('PINECONE_API_KEY')
google_api_key = userdata.get('GOOGLE_API_KEY')

if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = google_api_key


In [ ]:
# Import the Pinecone library
from pinecone import Pinecone
from pinecone import ServerlessSpec

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
# Create a dense index with integrated embedding
index_name = "online-web-project"
existing_indexes = [index.name for index in pc.list_indexes()]
if index_name not in existing_indexes:
  pc.create_index(
    name=index_name,
    dimension=3072,
    metric="cosine",
    spec=ServerlessSpec(
      cloud="aws",
      region="us-east-1"
    )
  )

index = pc.Index(index_name)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vector = embeddings.embed_query("Obaid")
vector[:5]

[-0.009420379996299744,
 -0.004691897891461849,
 0.03775719553232193,
 -0.08167464286088943,
 -0.000533588114194572]

In [ ]:
from langchain_pinecone import Pinecone as PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['770308c0-f59e-4c0f-a9e6-8744799c5d0d',
 '41a31cbf-1b22-464d-a2c1-3720158bf7bf',
 'ea62ad00-3293-4a78-932a-c28c0bb6b2a9',
 '4c9b6ec8-f4b9-41dc-bfb4-24d82bcdb8b0',
 'e1e13fbf-b2df-4dde-a75e-d2749a8e79fd',
 'f63e18a8-3bb2-4cb5-b5ac-09e6836662eb',
 'aa4c84cd-7477-4dea-aa82-cf805146aab3',
 'ed90cad2-b430-40ea-8732-689c8dad73ba',
 '9d0f9b52-c5ed-40dd-a6bd-ff02d1a92074',
 'c8ec21e5-e4f8-4987-8cba-abcd0405f915']

In [ ]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.878494] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
client = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash-lite",
)

In [ ]:
def answer_to_user(query: str) -> str:
    vector_results = vector_store.similarity_search_with_score(query)

    answers = client.invoke(f"Answer to the query: {query} using references {vector_results}")
    print(f"Answer: {answers.content}")


answer_to_user("Will it be hot tomorrow?")

Answer: No, it will not be hot tomorrow. The weather forecast indicates it will be cloudy and overcast with a high of 62 degrees.
